In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
np.set_printoptions(precision=6, suppress=True, linewidth=280)

from tqdm import trange
import torch
from muzero import Muzero
from environments import TicTacToe
from mcts import Tree
from network import Network
import random

my_seed = 1954

In [2]:
np.random.seed(my_seed)

M = Muzero(TicTacToe) #creating the agent
M_random = Muzero(TicTacToe) #creating the random adversary

game = TicTacToe()

nb_opti_steps = 100
nb_cycles = 30 #I'm open to changing names, wasn't sure how to call it otherwise
nb_leaves_per_move = 30
nb_episodes = 100
nb_games = 100

In [ ]:
np.random.seed(my_seed)

#traininig model
for _ in range(nb_cycles):
    M.REPLAY_BUFFER = []
    
    for _ in trange(nb_episodes): #playing
        with torch.no_grad():
            M.REPLAY_BUFFER.append(M.mcts.full_episode(nb_leaves_per_move))
            
    for _ in trange(nb_opti_steps): #optimisation 
        M.optimize_step()
    M.network.save('./models/ttt_candidate1.ptc') 

  0%|          | 0/100 [00:00<?, ?it/s]

Check Win:  False False False
Check Win:  False False False
Check Win:  False False False
Check Win:  False False False
Reward  0
Check Win:  False False False
Check Win:  False False False
Play  0 
 [[0 0 0]
 [0 0 0]
 [0 0 0]]
Check Win:  False False False
Check Win:  False False False
Reward  0
Check Win:  False False False
Check Win:  False False False
Check Win:  False False False
Check Win:  False False False
Reward  0
Check Win:  False False False
Check Win:  False False False
Check Win:  False False False
Check Win:  False False False
Check Win:  False False False
Check Win:  False False False
Reward  0
Check Win:  False False False
Check Win:  False False False
Play  0 
 [[0 0 0]
 [0 0 1]
 [0 0 0]]
Check Win:  False False False
Check Win:  False False False
Reward  0
Check Win:  False False False
Check Win:  False False False
Check Win:  False False False
Check Win:  False False False
Reward  0
Check Win:  False False False
Check Win:  False False False
Check Win:  False False 

  1%|          | 1/100 [00:00<00:38,  2.60it/s]

Check Win:  False False False
Check Win:  False False False
Check Win:  False False False
Check Win:  False False False
Reward  0
Check Win:  False False False
Check Win:  False False False
Play  0 
 [[0 0 0]
 [0 0 0]
 [0 0 0]]
Check Win:  False False False
Check Win:  False False False
Reward  0
Check Win:  False False False
Check Win:  False False False
Check Win:  False False False
Check Win:  False False False
Reward  0
Check Win:  False False False
Check Win:  False False False
Check Win:  False False False
Check Win:  False False False
Check Win:  False False False
Check Win:  False False False
Reward  0
Check Win:  False False False
Check Win:  False False False
Play  0 
 [[0 0 0]
 [0 0 0]
 [0 0 1]]
Check Win:  False False False
Check Win:  False False False
Reward  0
Check Win:  False False False
Check Win:  False False False
Check Win:  False False False
Check Win:  False False False
Reward  0
Check Win:  False False False
Check Win:  False False False
Check Win:  False False 

  2%|▏         | 2/100 [00:00<00:36,  2.65it/s]

...

Check Win:  False False False
Check Win:  False False False
Check Win:  False False False
Check Win:  False False False
Reward  0
Check Win:  False False False
Check Win:  False False False
Play  0 
 [[0 0 0]
 [0 0 0]
 [0 0 0]]
Check Win:  False False False
Check Win:  False False False
Reward  0
Check Win:  False False False
Check Win:  False False False
Check Win:  False False False
Check Win:  False False False
Reward  0
Check Win:  False False False
Check Win:  False False False
Check Win:  False False False
Check Win:  False False False
Check Win:  False False False
Check Win:  False False False
Reward  0
Check Win:  False False False
Check Win:  False False False
Play  0 
 [[0 0 0]
 [0 1 0]
 [0 0 0]]
Check Win:  False False False
Check Win:  False False False
Reward  0
Check Win:  False False False
Check Win:  False False False
Check Win:  False False False
Check Win:  False False False
Reward  0
Check Win:  False False False
Check Win:  False False False
Check Win:  False False 

 68%|██████▊   | 68/100 [00:32<00:15,  2.11it/s]

Check Win:  False False False
Check Win:  False False False
Check Win:  False False False
Check Win:  False False False
Reward  0
Check Win:  False False False
Check Win:  False False False
Play  0 
 [[0 0 0]
 [0 0 0]
 [0 0 0]]
Check Win:  False False False
Check Win:  False False False
Reward  0
Check Win:  False False False
Check Win:  False False False
Check Win:  False False False
Check Win:  False False False
Reward  0
Check Win:  False False False
Check Win:  False False False
Check Win:  False False False
Check Win:  False False False
Check Win:  False False False
Check Win:  False False False
Reward  0
Check Win:  False False False
Check Win:  False False False
Play  0 
 [[0 0 0]
 [0 1 0]
 [0 0 0]]
Check Win:  False False False
Check Win:  False False False
Reward  0
Check Win:  False False False
Check Win:  False False False
Check Win:  False False False
Check Win:  False False False
Reward  0
Check Win:  False False False
Check Win:  False False False
Check Win:  False False 

 69%|██████▉   | 69/100 [00:33<00:14,  2.16it/s]

Check Win:  False False False
Check Win:  False False False


In [ ]:
#Save models
np.random.seed(my_seed)

M.network.save('./models/m_201125_a.ptc')
M_random.network.save('./models/random.ptc')

In [ ]:
np.random.seed(my_seed)

def play_a_game(player1, player2):
    """
    Returns the reward for player 1, since it's + if P1 wins and - if P2 wins
    """
    game = TicTacToe()
    player1.environment = game
    player2.environment = game
    
    while True: #while no one wins and it's not a draw
        
        print("PLAYER 1: ", flush=True)
        game.play(player1.move(game.state, leaves_per_move=20)) #P1 does one move
        if game.end: #if the game is over i.e. P1 wins or it's a draw
            game.show
            return game.reward #return the reward associated to the game
        #Here the game is not yet over, it's P2's turn
        print("PLAYER 2: ", flush=True)
        game.play(player2.move(game.state, leaves_per_move=20)) #P2 does one move
        if game.end:
            game.show #if the game is over i.e. P1 wins or it's a draw
            return -game.reward #return the reward associated to the game

In [ ]:
#MuZero agent vs random
np.random.seed(my_seed)

net1 = Network(TicTacToe.num_observations)
net1.load('models/ttt_candidate1.ptc')

net2 = Network(TicTacToe.num_observations)
net2.load('models/random.ptc')

with torch.no_grad():
    player = Tree(net1, game)
    random = Tree(net2, game)

with torch.no_grad():
    score_white_a = 0
    for g in (t:=trange(nb_games)):
        score_white_a += play_a_game(player, random)
        t.set_description(f"Score as white: {score_white_a}")

with torch.no_grad():
    score_black_a = 0
    for g in (t:=trange(nb_games)):
        score_black_a -= play_a_game(random, player)
        t.set_description(f"Score as black: {score_black_a}")

print(f'Final Scores = W: {score_white_a} / B: {score_black_a} over {nb_games} games')

In [ ]:
#random agent vs random
np.random.seed(my_seed)

net1 = Network(TicTacToe.num_observations)
net1.load('models/random.ptc')

net2 = Network(TicTacToe.num_observations)
net2.load('models/random.ptc')


with torch.no_grad():
    player = Tree(net1, game)
    random = Tree(net2, game)

with torch.no_grad():
    score_white_b = 0
    for g in (t:=trange(nb_games)):
        score_white_b += play_a_game(player, random)
        t.set_description(f"Score as white: {score_white_b}")

with torch.no_grad():
    score_black_b = 0
    for g in (t:=trange(nb_games)):
        score_black_b -= play_a_game(random, player)
        t.set_description(f"Score as black: {score_black_b}")

print(f'Final Scores = W: {score_white_b} / B: {score_black_b} over {nb_games} games')

In [ ]:
#MuZ vs MuZ
np.random.seed(my_seed)

net1 = Network(TicTacToe.num_observations)
net1.load('models/ttt_candidate1.ptc')

net2 = Network(TicTacToe.num_observations)
net2.load('models/ttt_candidate1.ptc')

with torch.no_grad():
    player = Tree(net1, game)
    random = Tree(net2, game)

with torch.no_grad():
    score_white_c = 0
    for g in (t:=trange(nb_games)):
        score_white_c += play_a_game(player, random)
        t.set_description(f"Score as white: {score_white_c}")

with torch.no_grad():
    score_black_c = 0
    for g in (t:=trange(nb_games)):
        score_black_c -= play_a_game(random, player)
        t.set_description(f"Score as black: {score_black_c}")

print(f'Final Scores = W: {score_white_c} / B: {score_black_c} over {nb_games} games')

In [ ]:
print(f"Score comparison over {nb_games} games \n MuZero vs Random = W: {score_white_a} & B: {score_black_a} \n Random vs Random = W: {score_white_b} & B: {score_black_b} \n MuZero vs MuZero = W: {score_white_c} & B: {score_black_c}")